# NISQ Experiment

In [1]:
import sys
sys.path.insert(0, '../../src_numpy/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit import Aer
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator
from copy import deepcopy

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from utils import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
#qk.IBMQ.save_account("66718f8f8aef22bcb6ebe86ad94a11f1fd1f4c55100829bb13f16e6b448e0a1ec6d09c459d738f58d0cbd8398a2a1f5e185a4706a61b6f896a5ce2983e136429", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibm_oslo")

## Pauli String Expectation Values

### One Qubits

In [3]:
n = 1
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)

np.random.seed(42)
random.seed(42)

state_index, observ_index = index_generator(n)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):
    
    config = numberToBase(i, 6, n)
    state = prepare_input(config)
    state_circuit = prepare_input(config, return_mode = "circuit")

    config = numberToBase(j, 4, n)
    observable = pauli_observable(config)
    observable_circuit = pauli_observable(config, return_mode = "circuit")
    
    input_list.append([state, observable])
    circuit = state_circuit
    circuit.barrier()
    circuit = circuit.compose(circuit_target)
    circuit.barrier()
    circuit.add_register(observable_circuit.cregs[0])
    circuit = circuit.compose(observable_circuit)
    
    circuit_list.append(circuit)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        for k in range(5):
            circuit.h(0)
            circuit.barrier()
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)
    
#result_list = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0).result()
#counts_list = [result_list.get_counts(circuit) for circuit in circuit_list]

#expectation_list = [expected_parity(counts) for counts in counts_list[:18]]
#expectation_list.extend([counts["0"]/20000 for counts in counts_list[18:]]) 
#data = [input_list, expectation_list]

#pickle.dump(data, open("..\..\data\oneQubits_expectation.p", "wb"))

In [4]:
job = backend.retrieve_job("62b33d8b8d45c1a0f627d79e")
result_list = job.result()
counts_list = result_list.get_counts()

expectation_list = [expected_parity(counts) for counts in counts_list[:18]]
expectation_list.extend([counts["0"]/20000 for counts in counts_list[18:]]) 
data = [input_list, expectation_list]

pickle.dump(data, open("..\..\data\oneQubits_expectation.p", "wb"))

## Two Qubits

In [5]:
n = 2
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)

np.random.seed(42)
random.seed(42)

N = 324
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=False)

for i in range(14):
    circuit_list.extend(generate_bitstring_circuits(n))
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        for k in range(4):
            circuit.h(0)
            circuit.s(1)
            circuit.cnot(0,1)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)

    
#job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0)
#job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0)
#job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0)
#job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0)

#counts_list1 = [job1.result().get_counts(circuit) for circuit in circuit_list[:100]]
#counts_list2 = [job2.result().get_counts(circuit) for circuit in circuit_list[100:200]]
#counts_list3 = [job3.result().get_counts(circuit) for circuit in circuit_list[200:300]]
#counts_list4 = [job4.result().get_counts(circuit) for circuit in circuit_list[300:]]

#counts_list = counts_list1 + counts_list2 + counts_list3 + counts_list4
#data = [input_list, counts_list]

#pickle.dump(data, open("..\..\data\twoQubits_expectation_povm.p", "wb"))

In [3]:
job1 = backend.retrieve_job("62b7759e6a05a559c68dc29b")
job2 = backend.retrieve_job("62b775a4b3f3485de6f24723")
job3 = backend.retrieve_job("62b775a9cd175bbf70e6ef90")
job4 = backend.retrieve_job("62b775afa8fe828a292c3185")

In [6]:
counts_list1 = [job1.result().get_counts(circuit) for circuit in circuit_list[:100]]
counts_list2 = [job2.result().get_counts(circuit) for circuit in circuit_list[100:200]]
counts_list3 = [job3.result().get_counts(circuit) for circuit in circuit_list[200:300]]
counts_list4 = [job4.result().get_counts(circuit) for circuit in circuit_list[300:]]

counts_list = counts_list1 + counts_list2 + counts_list3 + counts_list4
data = [input_list, counts_list]

pickle.dump(data, open("..\..\data\twoQubits_expectation_povm.p", "wb"))

## Three Qubits Error Correction, QFT

In [4]:
def qft(n):
    circuit = qk.QuantumCircuit(n)
    for i in range(n):
        circuit.h(i)
        for j in range(i+1, n):
            circuit.cp(2*np.pi/2**(j-i+1), j, i)
            
    return circuit

In [5]:
n = 3
circuit_target = qft(n)

np.random.seed(42)
random.seed(42)

np.random.seed(42)
random.seed(42)

N = 500 - 64
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=False)

for i in range(8):
    circuit_list.extend(generate_bitstring_circuits(n))

job_list = []
for i in tqdm(range(5)):
    job = qk.execute(circuit_list[i*100:(i+1)*100], backend, shots = 20000, optimization_level = 0)
    job_list.append(job)
    

counts_list = []
for i, job in enumerate(job_list):
    result = job.result()
    counts_list.extend([result.get_counts(circuit) for circuit in circuit_list[i*100:(i+1)*100]])

data = [input_list, counts_list]

pickle.dump(data, open("..\..\data\threeQubits_expectation_qft_error.p", "wb"))

  0%|          | 0/5 [00:00<?, ?it/s]

## One Qubits with Error Correction

In [3]:
n = 1
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.barrier()
circuit_target.h(0)

np.random.seed(42)
random.seed(42)

state_index, observ_index = index_generator(n)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):
    
    config = numberToBase(i, 6, n)
    state = prepare_input(config)
    state_circuit = prepare_input(config, return_mode = "circuit")

    config = numberToBase(j, 4, n)
    observable = pauli_observable(config)
    observable_circuit = pauli_observable(config, return_mode = "circuit")
    
    input_list.append([state, observable])
    circuit = state_circuit
    circuit.barrier()
    circuit = circuit.compose(circuit_target)
    circuit.barrier()
    circuit.add_register(observable_circuit.cregs[0])
    circuit = circuit.compose(observable_circuit)
    
    circuit_list.append(circuit)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        for k in range(5):
            circuit.h(0)
            circuit.barrier()
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)
    
circuit_list.extend(generate_bitstring_circuits(n))
    
#result_list = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0).result()
#counts_list = [result_list.get_counts(circuit) for circuit in circuit_list]

#data = [input_list, counts_list]
#pickle.dump(data, open("..\..\data\oneQubits_expectation_error.p", "wb"))

In [4]:
result_list = backend.retrieve_job("62b4ac620133390082ddcec4").result()
counts_list = [result_list.get_counts(circuit) for circuit in circuit_list]

data = [input_list, counts_list]
pickle.dump(data, open("..\..\data\oneQubits_expectation_error.p", "wb"))

## Two Qubit Error Correction With Trace

In [4]:
n = 2
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)

np.random.seed(42)
random.seed(42)

N = 500 - 16 - 20
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=True)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        for k in range(4):
            circuit.h(0)
            circuit.s(1)
            circuit.cnot(0,1)
            circuit.barrier()
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)

for i in range(4):
    circuit_list.extend(generate_bitstring_circuits(n))

job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0)
job5 = qk.execute(circuit_list[400:],    backend, shots = 20000, optimization_level = 0)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
result5 = job5.result()

counts_list = []
counts_list.extend([result1.get_counts(circuit) for circuit in circuit_list[:100]])
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])
counts_list.extend([result5.get_counts(circuit) for circuit in circuit_list[400:]])

data = [input_list, counts_list]

pickle.dump(data, open("..\..\data\twoQubits_expectation_POVM_trace.p", "wb"))

## Two Qubit Error Correction With No Trace

In [3]:
n = 2
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.barrier()
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)

np.random.seed(42)
random.seed(42)

N = 324
input_list, circuit_list = generate_pauli_circuits(circuit_target, N, trace=False)
    
for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        for k in range(4):
            circuit.h(0)
            circuit.s(1)
            circuit.cnot(0,1)
            circuit.barrier()
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)

for i in range(4):
    circuit_list.extend(generate_bitstring_circuits(n))

job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0)
job4 = qk.execute(circuit_list[300:], backend, shots = 20000, optimization_level = 0)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()

counts_list = []
counts_list.extend([result1.get_counts(circuit) for circuit in circuit_list[:100]])
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:]])

data = [input_list, counts_list]

pickle.dump(data, open("..\..\data\twoQubits_expectation_POVM_notrace.p", "wb"))

## Three Qubit POVM

In [3]:
def variational_circuit(n):
    theta = np.random.uniform(-np.pi, np.pi, 4*n)
    circuit = qk.QuantumCircuit(n)
    for i, angle in enumerate(theta[:n]):
        circuit.ry(angle, i)
    
    for i, angle in enumerate(theta[n:2*n]):
        circuit.crx(angle, i, (i+1)%n)
        
    for i, angle in enumerate(theta[2*n:3*n]):
        circuit.ry(angle, i)
        
    for i, angle in enumerate(theta[3*n:]):
        circuit.crx(angle, (n-i)%n, n-i-1)
    
    return circuit

In [4]:
print(variational_circuit(2))

     ┌──────────────┐               ┌────────────┐ ┌─────────────┐ »
q_0: ┤ Ry(-0.78749) ├───────■───────┤ Rx(1.5034) ├─┤ Ry(-1.5963) ├─»
     └┬────────────┬┘┌──────┴──────┐└─────┬──────┘┌┴─────────────┴┐»
q_1: ─┤ Ry(1.0057) ├─┤ Rx(-1.0275) ├──────■───────┤ Ry(0.0018982) ├»
      └────────────┘ └─────────────┘              └───────────────┘»
«                     ┌────────────┐
«q_0: ───────■────────┤ Rx(2.9371) ├
«     ┌──────┴───────┐└─────┬──────┘
«q_1: ┤ Rx(-0.91048) ├──────■───────
«     └──────────────┘              


In [7]:
n = 2
np.random.seed(42)
random.seed(42)

circuit_target1 = variational_circuit(n)
circuit_target2 = variational_circuit(n)


N = 161
input_list, circuit_list   = generate_pauli_circuits(circuit_target1, N, trace=False)
input_list2, circuit_list2 = generate_pauli_circuits(circuit_target2, N, trace=False)
input_list3, circuit_list3 = generate_pauli_circuits(circuit_target1.compose(circuit_target2), N, trace=False)


input_list.extend(input_list2)
input_list.extend(input_list3)
circuit_list.extend(circuit_list2)
circuit_list.extend(circuit_list3)

for i in range(4):
    circuit_list.extend(generate_bitstring_circuits(n))

job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0)
job5 = qk.execute(circuit_list[400:], backend, shots = 20000, optimization_level = 0)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
result5 = job5.result()

counts_list = []
counts_list.extend([result1.get_counts(circuit) for circuit in circuit_list[:100]])
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])
counts_list.extend([result5.get_counts(circuit) for circuit in circuit_list[400:]])

data = [input_list, counts_list]

pickle.dump(data, open("..\..\data\twoQubits_expectation_variational.p", "wb"))